In [1]:
from sympy import *
import numpy as np

In [2]:
def gradient_descent(f, sym, x0, alpha = 0.1, eps = 1e-4):
    df = np.array([100])
    i = 1
    it = 1
    while eps < np.linalg.norm(df):
        df = np.array([f.diff(sym[0]).subs(zip(sym, x0)).evalf(), f.diff(sym[1]).subs(zip(sym, x0)).evalf()]).astype(np.float64)
        y = x0 - alpha*df
        if f.subs(zip(sym,y)).evalf()<f.subs(zip(sym, x0)).evalf():
            x0 = y
        else:
            alpha = alpha/2
        i = i + 4
        it = it + 1
    return f.subs(zip(sym,x0)).evalf(), x0, i, it

def gradient_descent(f, sym, x0, alpha = 0.1, eps = 1e-4):
    df = np.array([100,133])
    i = 1
    it = 1
    x0 = ImmutableDenseNDimArray(x0)
    while eps < df[0]**2 + df[1]**2:
        df = tensor.derive_by_array(f,sym).subs(zip(sym,x0))
        y = x0 - df.applyfunc(lambda x: x*alpha)
        if f.subs(zip(sym,y)).evalf() < f.subs(zip(sym, x0)).evalf():
            x0 = y
        else:
            alpha = alpha/2
        i = i + 4
        it = it + 1
    return f.subs(zip(sym,x0)).evalf(), x0, i, it

In [3]:
def iterative_search(f, sym, a, b, eps=1e-8, delta = 1000):
    eps = eps
    x0 = a
    x1 = a+delta
    it = 0
    while eps<abs(delta):
        x1 = a+delta
        while f.subs({sym : x1}).evalf() < f.subs({sym : x0}).evalf() and x1>a and x1<b:
            x0 = x1
            x1 = x1+delta
            it = it + 1
        delta = -delta/4
    return f.subs({sym : x0}).evalf(), x0, it

In [34]:
def newton_search(f, sym, eps=1e-10):
    x0 = iterative_search(f,sym,0,oo,1)[1]
    it = 0 
    d = diff(f, sym).subs({sym : x0}).evalf()
    while eps < abs(d):
        x0 = x0 - d/diff(f, sym, 2).subs({sym : x0}).evalf()
        it = it + 2
        if it>200:
            return iterative_search(f,sym,0,oo)
        d = diff(f, sym).subs({sym : x0}).evalf()
    return f.subs({sym : x0}).evalf(), x0, it

In [30]:
def golden_split_search(f, sym, a, b, eps=1e-8):
    tau = (np.sqrt(5)-1)/2
    x2 = a + tau*(b-a)
    x1 = a+b-x2
    f1 = f.subs({sym: x1})
    f2 = f.subs({sym: x2})
    t = 0
    while eps < (b-a)/2: 
        if f.subs({sym: x1})<=f.subs({sym: x2}):
            b = x2
            x2 = x1
            x1 = a+b-x1
        else:
            a = x1
            x1 = x2
            x2 = a+b-x2
        t = t+1
    return f.subs({sym: (a+b)/2}), (a+b)/2, t

In [6]:
def full_descent(f, sym, x0, eps=1e-4, minimizer = newton_search):
    theta = symbols('theta')
    i = 0
    it = 0
    df = np.array([f.diff(sym[0]).subs(zip(sym, x0)).evalf(), f.diff(sym[1]).subs(zip(sym, x0)).evalf()]).astype(np.float64)
    while eps < np.linalg.norm(df):
        df = np.array([f.diff(sym[0]).subs(zip(sym, x0)).evalf(), f.diff(sym[1]).subs(zip(sym, x0)).evalf()]).astype(np.float64)
        f_ = f.subs(zip(sym, x0-theta*df))
        theta_min = minimizer(f_, theta)
        x0 = x0 - theta_min[1]*df
        i = i + 2 + theta_min[2]
        it = it + 1
    return f.subs(zip(sym,x0)).evalf(), x0, i, it

In [7]:
def ortogonal_descent(f, sym, x0, eps=1e-4, restart = 0, minimizer = lambda x,y: newton_search(x,y), beta = 'v2'):
    theta = symbols('theta')
    i = 0
    it = 0
    df = np.array([f.diff(sym[0]).subs(zip(sym, x0)).evalf(16), f.diff(sym[1]).subs(zip(sym, x0)).evalf(16)]).astype(np.float64)
    df_1 = 0     
    while eps < np.linalg.norm(df):
        y = df-df_1
        theta_min = minimizer(f.subs(zip(sym, x0-theta*df)), theta)
        x0 = x0 - theta_min[1]*df
        i = i + 2 + theta_min[2]
        s = np.array([f.diff(sym[0]).subs(zip(sym, x0)).evalf(16), f.diff(sym[1]).subs(zip(sym, x0)).evalf(16)]).astype(np.float64)
    
        if beta == 'v3':
            b = (np.linalg.norm(s)/np.linalg.norm(df))**2
        elif beta == 'v1':
            b = float(re(np.dot(y,s)/np.dot(y, df)))
        elif beta == 'v2':
            b = float(np.dot(y,s)/(np.linalg.norm(df)**2))
        else:
            raise Exception("Unknown beta type")
        
        df_1 = df
        df = np.array(-1*s + df*b)
        it = it+1
        if restart and it%restart == 0:
            df = np.array([f.diff(sym[0]).subs(zip(sym, x0)).evalf(16), f.diff(sym[1]).subs(zip(sym, x0)).evalf(16)]).astype(np.float64)
    return f.subs(zip(sym, x0)).evalf(16), x0, i, it

# Задание 2

In [8]:
x1 = symbols('x1')
x2 = symbols('x2')
theta = symbols('theta')

def f(a):
    return x1**2+a*x2**2

In [9]:
a = 1
eps = 1e-3
gradient_descent(f(a), [x1,x2], [1,-5], eps=eps), full_descent(f(a), [x1,x2], [1,-5], eps=eps),\
ortogonal_descent(f(a), [x1,x2], [1,-5], eps=eps)

((1.20423772806809e-7, array([ 6.80564734e-05, -3.40282367e-04]), 173, 44),
 (0, array([0, 0], dtype=object), 6, 2),
 (0, array([0, 0], dtype=object), 4, 1))

In [10]:
a = 1
eps = 1e-5
gradient_descent(f(a), [x1,x2], [1,-5], eps=eps), full_descent(f(a), [x1,x2], [1,-5], eps=eps),\
ortogonal_descent(f(a), [x1,x2], [1,-5], eps=eps)

((1.02445216110626e-11, array([ 6.27710174e-07, -3.13855087e-06]), 257, 65),
 (0, array([0, 0], dtype=object), 6, 2),
 (0, array([0, 0], dtype=object), 4, 1))

In [11]:
a = 250
eps = 1e-3
gradient_descent(f(a), [x1,x2], [1,-5], eps=eps), full_descent(f(a), [x1,x2], [1,-5], eps=eps),\
ortogonal_descent(f(a), [x1,x2], [1,-5], eps=eps)

((2.44849917188919e-7,
  array([ 4.94823117e-004, -2.22698617e-303]),
  4877,
  1220),
 (3.96484493028951e-12,
  array([2.51847911440165e-8, -1.25923955728599e-7], dtype=object),
  16,
  4),
 (9.947397293725336e-8,
  array([0.000315394947144790, 1.00556118187421e-9], dtype=object),
  20,
  5))

In [12]:
a = 250
eps = 1e-5
gradient_descent(f(a), [x1,x2], [1,-5], eps=eps), full_descent(f(a), [x1,x2], [1,-5], eps=eps),\
ortogonal_descent(f(a), [x1,x2], [1,-5], eps=eps)

((2.46458570399349e-11,
  array([ 4.96445939e-006, -4.94065646e-324]),
  7813,
  1954),
 (4.40860063025847e-19,
  array([5.87907466763093e-10, -1.95166465920813e-11], dtype=object),
  22,
  6),
 (3.478776037713882e-14,
  array([1.86514771066129e-7, 1.48497590613481e-12], dtype=object),
  34,
  9))

In [13]:
a = 1000
eps = 1e-3
gradient_descent(f(a), [x1,x2], [1,-5], 2, eps=eps), full_descent(f(a), [x1,x2], [1,-5], eps=eps),\
ortogonal_descent(f(a), [x1,x2], [1,-5], eps=eps)

((2.48888469898506e-7, array([4.98887232e-04, 4.09995923e-81]), 15601, 3901),
 (6.34821691553673e-14,
  array([1.59348209035807e-9, -7.96741049339087e-9], dtype=object),
  16,
  4),
 (6.353819188262419e-9,
  array([7.97108473672733e-5, 1.59275139572528e-11], dtype=object),
  20,
  5))

In [14]:
a = 1000
eps = 1e-5
gradient_descent(f(a), [x1,x2], [1,-5], eps=eps), full_descent(f(a), [x1,x2], [1,-5], eps=eps),\
ortogonal_descent(f(a), [x1,x2], [1,-5], eps=eps)

((2.49049400608615e-11,
  array([4.99048495e-006, 4.94065646e-324]),
  31257,
  7815),
 (1.48849855997168e-21,
  array([3.73098892547821e-11, -3.10597366002320e-13], dtype=object),
  22,
  6),
 (1.419290370247580e-16,
  array([1.19133973739383e-8, 5.94956156887508e-15], dtype=object),
  34,
  9))

# Задание 3

In [15]:
f = 64*x1**2 + 126*x1*x2 + 64*x2**2 + -10*x1+30*x2 + 13

In [16]:
eps = 1e-3
gradient_descent(f, [x1,x2], [1,-5], eps=eps), full_descent(f, [x1,x2], [1,-5], eps=eps),\
ortogonal_descent(f, [x1,x2], [1,-5], eps=eps)

((-187.393700546578, array([  9.96028292, -10.03902307]), 3169, 793),
 (-187.393700787105,
  array([9.96062522772611, -10.0393674391430], dtype=object),
  48,
  12),
 (-187.3937007173045,
  array([9.96044283315183, -10.0391827545429], dtype=object),
  164,
  41))

# Задание 5

In [17]:
f = 100*(x1**2-x2)**2 + (x1-1)**2
eps = 1e-3
ortogonal_descent(f, [x1,x2], [-1,1], eps=eps)

(1.193056372415401e-6,
 array([0.998908794086180, 0.997813956013469], dtype=object),
 13324,
 2353)

In [18]:
eps = 1e-5
ortogonal_descent(f, [x1,x2], [-1,1], eps=eps)

(1.193998338136502e-10,
 array([0.999989083618588, 0.999978119143204], dtype=object),
 20676,
 4185)

# Задание 6

In [19]:
eps = 1e-3
ortogonal_descent(f, [x1,x2], [-1, 1], eps=eps, restart=1)

(9.967597860275690e-7,
 array([0.999002872845824, 0.998001742730548], dtype=object),
 22378,
 4445)

In [20]:
ortogonal_descent(f, [x1,x2],  [-1, 1], eps=eps,restart=2)

(6.342066569387720e-8,
 array([1.00025165359826, 1.00050432515780], dtype=object),
 1144,
 200)

In [21]:
eps = 1e-3
ortogonal_descent(f, [x1,x2],  [-1, 1], eps=eps,restart=3)

(1.024763788296749e-6,
 array([1.00101159685222, 1.00202800595976], dtype=object),
 1718,
 258)

In [22]:
eps = 1e-3
ortogonal_descent(f, [x1,x2],  [-1, 1], eps=eps,restart=4)

(1.197704386607404e-6,
 array([0.998906306496894, 0.997809886267641], dtype=object),
 30266,
 5114)

In [23]:
eps = 1e-3
ortogonal_descent(f, [x1,x2],  [-1, 1], eps=eps,restart=5)

(1.064910363312655e-6,
 array([0.998968635967277, 0.997934873572785], dtype=object),
 2268,
 345)

# Задание 7

In [24]:
f = 64*x1**2 + 126*x1*x2 + 64*x2**2 + -10*x1+30*x2 + 13
ortogonal_descent(f, [x1,x2], [2, 4], eps=eps, restart=2, minimizer=lambda x,y: iterative_search(x,y,0,oo,1e-4))

(-187.3937006050728, array([  9.96032724, -10.03906926]), 2906, 795)

In [25]:
ortogonal_descent(f, [x1,x2], [2, 4], eps=eps, restart=2, minimizer=lambda x,y: iterative_search(x,y,0,oo,1e-6))

(-187.3937006050728, array([  9.96032724, -10.03906926]), 2906, 795)

In [26]:
ortogonal_descent(f, [x1,x2], [2, 4], eps=eps, restart=2, minimizer=lambda x,y: iterative_search(x,y,0,oo,1e-8))

(-187.3937006050710, array([  9.96032723, -10.03906926]), 2927, 795)

In [31]:
ortogonal_descent(f, [x1,x2],[2, 4], eps=eps, restart=2, minimizer=lambda x,y: golden_split_search(x,y,0,1000,1e-4))

(-187.3937005757780, array([  9.96030425, -10.03904525]), 4935, 141)

In [32]:
ortogonal_descent(f, [x1,x2], [2, 4], eps=eps, restart=2, minimizer=lambda x,y: golden_split_search(x,y,0,1000,1e-6))

(-187.3937005807584, array([  9.96030909, -10.03904815]), 1764, 42)

In [33]:
ortogonal_descent(f, [x1,x2], [2, 4], eps=eps, restart=2, minimizer=lambda x,y: golden_split_search(x,y,0,1000,1e-8))

(-187.3937007870372, array([  9.96061639, -10.03935679]), 3740, 85)